## connecting to mysql through ssh tunnel and a proxyhost

In [1]:
from sshtunnel import SSHTunnelForwarder
import os
import pandas as pd
import numpy as np
import csv
import mysql.connector
from mysql.connector.constants import ClientFlag
from secureconfig import SecureConfigParser, SecureConfigException, zeromem
import datetime
from IPython.display import display, HTML


### Connection credentials are stored in a secure config format with passwords encrypted

    configpath = os.path.expanduser('~/config.ini')

The master key is stored in a hidden directory. 

    key_path = os.path.expanduser('~/.keys/db.key')

The config.ini file can contain credentials for many environments and is organized like Microsoft Windows INI files with passwords being encrypted. This could be modified to encrypt usernames and ip addresses if needed.

### setup  config.ini file

    using an editor configure a section labeling each section using [section]
    add required attributes 
    
    [section]
    dbusername = <username>
    dbport = 3306
    dbhost = <dbhost>
    proxy_host = <proxyhost>
    proxy_port = 22
    proxy_user = <ssh user with passwordless login>
    dbschema = <default schema>
    dbpassword = 
    
    
    Run secrets_config_editor.py to add the encrypted secrets 
    ./secrets_config_editor.py set --section=<section> --config=~/config.ini --key=~/.keys/db.key



### Note:

MacOS RSA keys don't work with paramiko. The workaround is to create an RSA key on linux and copy to the mac and use for ssh_pkey

If you are getting strange errors from secureconfig this snippet to help debug


    import paramiko

    pkey='/Users/markgruenberg/keys/id_rsa'
    key=paramiko.RSAKey.from_private_key_file(pkey) 



In [ ]:
# use to view the config.ini file

! cat ~/config.ini

In [2]:
configpath = os.path.expanduser('~/config.ini')
key_path = os.path.expanduser('~/.keys/db.key')


scfg = SecureConfigParser.from_file(key_path)
scfg.read(configpath)
section = 'uweee_dev'

with SSHTunnelForwarder(
    (scfg.get(section, 'proxy_host'), int(scfg.get(section, 'proxy_port'))),
    ssh_username = scfg.get(section, 'proxy_user'),
    ssh_pkey="/Users/markgruenberg/keys/id_rsa",
    remote_bind_address = (scfg.get(section, 'dbhost'), int(scfg.get(section, 'dbport')))
) as tunnel:

    config = {
        'user': scfg.get(section, 'dbusername'),
        'password': scfg.get(section, 'dbpassword'),
        'host': tunnel.tunnel_bindings.keys()[0][0],
        'port': tunnel.tunnel_bindings.keys()[0][1],
        'database': scfg.get(section, 'dbschema'),
        'autocommit': True
    }
    print(tunnel.local_bind_port)

    cnx = mysql.connector.connect(**config)
    # zero out memory
    zeromem(config['password'])
    


cur = cnx.cursor(buffered=True)
cur.execute("show variables like 'autocommit'")
print(cur.fetchall())



57141
[(u'autocommit', u'ON')]


In [3]:
sql = """
WITH possible_fk_columns AS
    (
    SELECT table_schema, TABLE_NAME,column_name FROM information_schema.columns WHERE (column_name LIKE 'fk_%' OR column_name LIKE '%_id')
    )
SELECT for_table_schema, TABLE_NAME, column_name, fk_name, for_table, ref_table, for_columns, ref_columns
FROM possible_fk_columns p
LEFT JOIN dbadmin.foreign_keys f
    ON p.table_name=f.for_table AND p.column_name=for_columns AND p.table_schema=f.for_table_schema
WHERE
    f.for_table_schema='uweee'
"""

df = pd.read_sql(sql,cnx)


In [4]:
display(HTML(
    df[df['fk_name'].notnull()][['TABLE_NAME','COLUMN_NAME','ref_table','for_columns','ref_columns']]
    .sort_values('COLUMN_NAME')
    .head(20)
    .to_html()))

,TABLE_NAME,COLUMN_NAME,ref_table,for_columns,ref_columns
19,campaignmonths_clientpaymentsubscriptions,campaignmonth_id,campaign_months,campaignmonth_id,id
20,campaignmonths_clientpaymentsubscriptions,client_payment_subscription_id,client_payment_subscriptions,client_payment_subscription_id,id
0,addresses,fk_address_types,address_types,fk_address_types,id
29,clients,fk_addresses,addresses,fk_addresses,id
2,admin_notifications,fk_admin_notification_types,admin_notification_types,fk_admin_notification_types,id
4,admin_notifications,fk_admin_notification_types,admin_notification_types,fk_admin_notification_types,id
38,ftp_zip_campaign_month_historys,fk_art,arts,fk_art,id
8,art_files,fk_art_file_types,art_file_types,fk_art_file_types,id
9,art_files,fk_art_folders,art_folders,fk_art_folders,id
13,art_templates,fk_art_template_types,art_template_types,fk_art_template_types,id


In [ ]:
# Mac RSA keys formating doesn't work with paramiko
# The fix is to create a RSA key on linux and copy to the mac
# You can test the key using ...
import paramiko

pkey='/Users/markgruenberg/keys/id_rsa'
key=paramiko.RSAKey.from_private_key_file(pkey) 

In [ ]:
sql = """select table_name,column_name,column_type,data_type,CHARACTER_MAXIMUM_LENGTH 
from information_schema.columns where table_schema='uweee'"""

df = pd.read_sql(sql,cnx)

display(HTML(
    df[(df['DATA_TYPE']=='varchar') & 
       (df['CHARACTER_MAXIMUM_LENGTH'] > 60)][['TABLE_NAME','COLUMN_NAME','CHARACTER_MAXIMUM_LENGTH']]\
    .sort_values(by=['TABLE_NAME','CHARACTER_MAXIMUM_LENGTH'],ascending=[True,False])\
    .to_html()))
